## Import libraries

In [1]:
import tensorflow as tf

In [2]:
from assimilation import assimilation
from stackage import stackage
from chunckage import chunckage

In [3]:
import numpy as np

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import h5py
import os, sys

In [6]:
from CircleMaker import circle

In [7]:
import inception

## Functions

In [ ]:
def reshaping(list_input):
    L = np.array(list_input)
    L = L.reshape(L.size,-1)
    
    return L

In [ ]:
def normalise_01(image_data):
    stack = image_data
    image_data -= np.min(stack, axis=0)
    image_data /= (np.max(stack, axis=0) - np.min(stack, axis=0))
    return image_data

## Image processing

In [8]:
S2_patch_path, S2_patch_id, S2_patch_lulc = chunckage("Images/big_raster/","RGB",".tif",'Images/big_sample/','sample.gpkg','tr_RGB',1)

S2_grid_path, S2_grid_id = chunckage("Images/big_raster/","BNIR",".tif",'Images/big_grid/','grid.gpkg','g_RGB',0)

(238890.0, 288990.0, 9846480.0, 9871580.0)
(238890.0, 289090.0, 9846380.0, 9871680.0)


In [11]:
data_path = 'Images/big_raster/tr_RGB_patch/'
with h5py.File(os.path.join(data_path,"tr_Inception_RGB.hdf5"), mode="a") as f:              
    f["id_rgb"] = S2_patch_id                          
    f["lulc_rgb"] = S2_patch_lulc        
    f["patch_rgb_path"] = S2_patch_path

print("Data exported in %s"%(data_path))

data_path = 'Images/big_raster/g_RGB_patch/'
with h5py.File(os.path.join(data_path,"g_Inception_RGB.hdf5"), mode="a") as f:              
    f["id_rgb"] = S2_grid_id                        
    f["patch_rgb_path"] = S2_grid_path

print("Data exported in %s"%(data_path))

Data exported in Images/big_raster/tr_RGB_patch/
Data exported in Images/big_raster/g_RGB_patch/


## Data preprocessing

In [ ]:
f = h5py.File('Images/big_raster/tr_RGB_patch/tr_Inception_RGB.hdf5', 'r')
S2_patch_id = np.array(f["id_rgb"])
S2_patch_lulc = np.array(f["lulc_rgb"])
S2_patch_path = f["patch_rgb_path"]
                         
f = h5py.File('Images/big_raster/g_RGB_patch/g_Inception_RGB.hdf5', 'r')         
S2_grid_id = np.array(f["id_rgb"])                     
S2_grid_path = f["patch_rgb_path"]

In [ ]:
stack = stackage(S2_grid_path)
S2_stack_grid = normalise_01(stack)

In [ ]:
S2_patch_id = S2_patch_id.reshape(S2_patch_id.size,1)

S2_patch_lulc = S2_patch_lulc.reshape(S2_patch_lulc.size,1)

S2_grid_id = S2_grid_id.reshape(S2_grid_id.size,1)

### Dividing train and test patches

In [ ]:
patch_id_train, patch_id_test, patch_lulc_train, patch_lulc_test = train_test_split(S2_patch_id,S2_patch_lulc, test_size=0.2, random_state=100)
patch_id_train = reshaping(patch_id_train)
patch_id_test = reshaping(patch_id_test)
patch_lulc_train = reshaping(patch_lulc_train)
patch_lulc_test = reshaping(patch_lulc_test)  

In [ ]:
circle(patch_lulc_train,patch_lulc_test,'Repartition des données entrainements et de tests')

In [8]:
stack_train = assimilation(patch_id_train, patch_id_test, patch_lulc_test,S2_patch_path)[0]
stack_test = assimilation(patch_id_train, patch_id_test, patch_lulc_test,S2_patch_path)[1]

NameError: name 'patch_id_train' is not defined

## Model

In [8]:
tf.__version__

'2.9.1'

In [ ]:
tf.keras.applications.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(75,75,3),
    classes=1000
)

In [ ]:


model = InceptionV3(input_shape=(224,224,3), weights='imagenet', include_top=False)

In [ ]:



# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


history = model.fit(stack_train,
          patch_lulc_train, 
          validation_data=(stack_test, patch_lulc_test),
          epochs=100,
          batch_size=batch_size)

In [ ]:
predict = model.predict(S2_stack_grid)

In [ ]:
import os, signal

os.kill(os.getpid() , signal.SIGKILL)